<a href="https://colab.research.google.com/github/isaacyeos/nlp/blob/master/Toxic_Words_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use("ggplot")

import pandas as pd
import numpy as np

import tensorflow as tf

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.optimizers import Adam

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
train_file = "gdrive/My Drive/0-software_dev_notes/nlp/toxic-comment-classification/train_preprocessed.csv"
test_file = "gdrive/My Drive/0-software_dev_notes/nlp/toxic-comment-classification/test_preprocessed.csv"
test_labels = "gdrive/My Drive/0-software_dev_notes/nlp/toxic-comment-classification/test_labels.csv"

In [0]:
test_labels_df = pd.read_csv(test_labels).fillna("blank")
train_df = pd.read_csv(train_file).fillna("blank")
# train_df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = train_df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].astype(int)
# test_df = pd.read_csv(test_file).fillna("blank")
test_df = pd.read_csv(test_file)
test_df = test_df.combine_first(test_labels_df)

In [0]:
train_df.head(20)

,comment_text,id,identity_hate,insult,obscene,set,severe_toxic,threat,toxic,toxicity
0,explanation why the edits made under my userna...,0000997932d777bf,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
1,d aww he matches this background colour i m s...,000103f0d9cfb60f,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
2,hey man i m really not trying to edit war it...,000113f07ec002fd,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
3,more i can t make any real suggestions on im...,0001b41b1c6bb37e,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
4,you sir are my hero any chance you remember...,0001d958c54c6e35,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
5,congratulations from me as well use the tool...,00025465d4725e87,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
6,cock suck before you piss around on my work,0002bcb3da6cb337,0.0,1.0,1.0,train,1.0,0.0,1.0,4.0
7,your vandalism to the matt shirvington article...,00031b1e95af7921,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
8,sorry if the word nonsense was offensive to ...,00037261f536c51d,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
9,alignment on this subject and which are contra...,00040093b2687caa,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0


In [0]:
train_df.shape

(159571, 10)

In [0]:
test_df = test_df[test_df.identity_hate != -1.0]

In [0]:
test_df.head(20)

,comment_text,id,identity_hate,insult,obscene,set,severe_toxic,threat,toxic,toxicity
5,thank you for understanding i think very high...,0001ea8717f6de06,0.0,0.0,0.0,test,0.0,0.0,0.0,NaN
7,dear god this site is horrible,000247e83dcc1211,0.0,0.0,0.0,test,0.0,0.0,0.0,NaN
11,somebody will invariably try to add religion ...,0002f87b16116a7f,0.0,0.0,0.0,test,0.0,0.0,0.0,NaN
13,it says it right there that it is a type the...,0003e1cccfd5a40a,0.0,0.0,0.0,test,0.0,0.0,0.0,NaN
14,before adding a new product to the list make...,00059ace3e3e9a53,0.0,0.0,0.0,test,0.0,0.0,0.0,NaN
16,this other one from,000663aff0fffc80,0.0,0.0,0.0,test,0.0,0.0,0.0,NaN
17,reason for banning throwing this article need...,000689dd34e20979,0.0,0.0,0.0,test,0.0,0.0,0.0,NaN
19,block from editing wikipedia,000844b52dee5f3f,0.0,0.0,0.0,test,0.0,0.0,0.0,NaN
21,arabs are committing genocide in iraq but no...,00091c35fa9d0465,0.0,0.0,0.0,test,0.0,0.0,1.0,NaN
22,please stop if you continue to vandalize wiki...,000968ce11f5ee34,0.0,0.0,0.0,test,0.0,0.0,0.0,NaN


In [0]:
test_df.shape

(63978, 10)

In [0]:
train_df["comment_text"] = train_df["comment_text"].astype(str)
test_df["comment_text"] = test_df["comment_text"].astype(str)

In [0]:
X_train = train_df["comment_text"].values
y_train = train_df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values


In [0]:
X_train.shape

(159571,)

In [0]:
y_train[0]

array([0., 0., 0., 0., 0., 0.])

In [0]:
X_test = test_df["comment_text"].values
y_test = test_df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values


In [0]:
X_test.shape

(63978,)

In [0]:
y_test[23]

array([0., 0., 0., 0., 0., 0.])

In [0]:
train_df.dtypes

comment_text      object
id                object
identity_hate    float64
insult           float64
obscene          float64
set               object
severe_toxic     float64
threat           float64
toxic            float64
toxicity         float64
dtype: object

In [0]:
test_df.dtypes

comment_text      object
id                object
identity_hate    float64
insult           float64
obscene          float64
set               object
severe_toxic     float64
threat           float64
toxic            float64
toxicity         float64
dtype: object

In [0]:
max_words = 20000
max_len = 100

tok = Tokenizer(num_words=max_words)
# tok = Tokenizer()
# tok.fit_on_texts(X_train)
tok.fit_on_texts(list(X_train) + list(X_test))

train_sequences = tok.texts_to_sequences(X_train)
test_sequences = tok.texts_to_sequences(X_test)
X_train = sequence.pad_sequences(train_sequences,maxlen=max_len)
X_test = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [0]:
X_train.shape

(159571, 100)

In [0]:
X_test.shape

(63978, 100)

In [0]:
X_train[10]

array([   4,  262,  725,    9,    7,   36,   83,   24,    7,   20,  701,
         64, 1380,  402, 1346,    9,    7,   20, 1958,   92,  124,    5,
        860,   89,  164,    7,   36,  143,    6,  165,    4, 1380,    7,
         20,  701,   34,  429,   13,  170, 1188,    5, 4517,  347,   88,
         16,  148,   49,  713,  151,   46,   20,   58,  860,   18,  911,
         15,  413,   12,  277,  119,   24,    1,  121,    8, 1401,  182,
          6,  278,  200, 1098,  495,   29,  289,   83,   85,    1,  121,
         47,   16,  148,  793,  151,   24,    7,   20,   57,  251,   51,
        220,   89,   35,    1,  381,  262,  251,   30,  135,    7,   43,
        592], dtype=int32)

In [0]:
X_test[10]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,  1373,     3,    20,   597,     1,  1121,    73,
        1240,    10,   171,     9,  5990,   280,  2923,   107,    14,
          83,    57,   444,     4,  1373,    13,     8,    14,   379,
        5990,   280,  2923,  6980,    15,     1, 16840,  1373,     4,
           1,  3782,     9,     8,    93,   280, 14350,    13, 16840,
        1373,     8,    41,  2573,    10,     2,   454,   324,     5,
          34,  1356,  1037,  1963,     1, 15775,     1,   888,     8,
           9,  1258,   280,  2923,   209,   903,     1,  3038,    17,
        1373,    27, 18743,     2,  3150,     1,  3782,  1055,     1,
       15775], dtype=int32)

In [0]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, Embedding, GlobalMaxPooling1D
from tensorflow.keras.optimizers import RMSprop

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical


from tensorflow.keras.callbacks import EarlyStopping

In [0]:
Inp = Input(name='inputs',shape=[max_len])
x = Embedding(max_words,50,input_length=max_len)(Inp)
x = LSTM(64,name='LSTM_01')(x)
# x = Dense(256,activation='relu',name='Dense_01')(x)
x = Dropout(0.2,name='Dropout')(x)
out = Dense(6,activation='sigmoid', name='output')(x)


In [0]:
model = Model(inputs=Inp,outputs=out)

In [0]:
model.compile(loss='binary_crossentropy',optimizer=Adam(0.01),metrics=['accuracy'])

In [0]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 100)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 50)           1000000   
_________________________________________________________________
LSTM_01 (LSTM)               (None, 64)                29440     
_________________________________________________________________
Dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
output (Dense)               (None, 6)                 390       
Total params: 1,029,830
Trainable params: 1,029,830
Non-trainable params: 0
_________________________________________________________________


In [0]:
early_stop = EarlyStopping(monitor='val_loss',min_delta=0.0001)

In [0]:
model.fit(X_train,y_train,
          batch_size=128,
          epochs=10,
          validation_data=(X_test, y_test),
          callbacks=[early_stop])
# model.fit(X_train,y_train,
#           batch_size=128,
#           epochs=10,
#           validation_data=(X_test, y_test))

Epoch 1/10
1247/1247 [==============================] - 45s 36ms/step - loss: 0.0626 - accuracy: 0.9766 - val_loss: 0.0700 - val_accuracy: 0.9975
Epoch 2/10
1247/1247 [==============================] - 45s 36ms/step - loss: 0.0470 - accuracy: 0.9887 - val_loss: 0.0657 - val_accuracy: 0.9974
Epoch 3/10
1247/1247 [==============================] - 44s 36ms/step - loss: 0.0421 - accuracy: 0.9768 - val_loss: 0.0732 - val_accuracy: 0.9969
